Question 2
==

In [1]:
using Optim
using DataFrames
using ForwardDiff
using Roots
using Distributions

In [2]:
data = convert(Matrix,readtable("hsdata.csv"));
dataexp = exp(data);

In [3]:
#operates on single observations!
function g(w,z,theta) 
    return (theta[1] * w[1]^(-theta[2]) * w[2] - 1) .* collect(z[:])
end

#operates on data set!
function gn(w,z,theta)
    i = 1
    result = g(w[i,:],z[i,:],theta)
    
    for i in 2:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)
    end
    return result/size(w)[1]
end

#Wrapper creates a closure around the data set provided
function Qn_wrapper(w,z)
    return theta -> 1/2*(gn(w,z,theta)'*gn(w,z,theta))[1,1]
end

function Qn_wrapper(w,z,What)
    return theta -> 1/2*(gn(w,z,theta)'*What*gn(w,z,theta))[1,1]
end

#Calculates What
function What(w,z,theta) 
    result = zeros(Float64, size(z)[2], size(z)[2])
    for i in 1:size(w)[1]
        result .+= g(w[i,:],z[i,:],theta)*g(w[i,:],z[i,:],theta)'
    end
    return inv(result/size(z)[1])
end

#creates a closure around the data set
function gn_wrapper(w,z)
    return theta -> gn(w,z,theta)
end

gn_wrapper (generic function with 1 method)

In [4]:
#w = dataexp[2:end,[2,4]]
#z = hcat(ones(size(dataexp)[1]-1), dataexp[1:end-1,4], 1 .+ data[1:end-1,2])
w = convert(SharedArray{Float64},dataexp[2:end,[2,4]])
z = convert(SharedArray{Float64},hcat(ones(size(dataexp)[1]-1), dataexp[1:end-1,4], 1 .+ data[1:end-1,2]));
n = size(w)[1]

Qn = Qn_wrapper(w,z)
optres = optimize(Qn, [1., 4.], BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true));

In [5]:
θhat = optres.minimum
WhatA = What(w,z,θhat);

In [6]:
QnWhatA = Qn_wrapper(w,z,WhatA)
optresB = optimize(QnWhatA, θhat, BFGS(), Optim.OptimizationOptions(g_tol=1e-12, autodiff=true))
θhat = optresB.minimum

2-element Array{Float64,1}:
 0.978529
 3.386   

In [7]:
dgn(theta) = ForwardDiff.jacobian(gn_wrapper(w,z),theta)
GhatB = convert(Array{Float64,2},dgn(θhat))
aVar = inv(GhatB'*What(w,z,θhat)*GhatB)

2×2 Array{Float64,2}:
 0.10586    5.40968
 5.40968  321.828  

a
--
The Wald-test ist:

In [8]:
a(theta) = [theta[1]-0.99, theta[2]-4]
A(theta) = ForwardDiff.jacobian(a, theta)
ξW = n*(a(θhat)'*inv(A(θhat)*aVar*A(θhat)')*a(θhat))[1,1]

0.3137423217331219

In [9]:
1-cdf(Chisq(2), ξW)

0.8548141852421143

The LM-test is:

In [10]:
ξLM = (n*ForwardDiff.gradient(QnWhatA,[0.99, 4])'*(aVar)*ForwardDiff.gradient(QnWhatA,[0.99, 4]))[1,1]

0.22557851779285745

In [11]:
1-cdf(Chisq(2), ξLM)

0.893338903506397

The QLR-test is:

In [12]:
ξQLR = 2*n*(- QnWhatA(θhat) + QnWhatA([0.99, 4]))

0.2631771655274612

In [13]:
1-cdf(Chisq(2), ξQLR)

0.876701611046324

b
--
The Wald-test ist:

In [14]:
a(theta) = theta[1]-0.99
ξW = n*(a(θhat)'*inv(aVar[1,1])*a(θhat))[1,1]

0.3094975163248656

In [15]:
1-cdf(Chisq(2), ξW)

0.8566303718383055

The LM-test is:

In [16]:
gammaRes = optimize(gamma -> QnWhatA([0.99, gamma]), 2,6).minimum
ξLM = (n*ForwardDiff.gradient(QnWhatA,[0.99, gammaRes])'*(aVar)*ForwardDiff.gradient(QnWhatA,[0.99, gammaRes]))[1,1]

0.2204654210213198

In [17]:
1-cdf(Chisq(2), ξLM)

0.8956256895321376

The QLR-test is:

In [18]:
ξQLR = 2*n*(- QnWhatA(θhat) + QnWhatA([0.99, gammaRes]))

0.25775040434650043

In [19]:
1-cdf(Chisq(2), ξQLR)

0.8790836664313266

Question 3
==
a
--
I fixed my wrong sample correction from PS3:

In [20]:
θhat

2-element Array{Float64,1}:
 0.978529
 3.386   

In [21]:
θhat[1] .+ sqrt(aVar[1,1]/n) * [-1.96, 1.96]

2-element Array{Float64,1}:
 0.938116
 1.01894 

In [22]:
θhat[2] .+ sqrt(aVar[2,2]/n) * [-1.96, 1.96]

2-element Array{Float64,1}:
 1.15773
 5.61427

b
--

In [23]:
#all functions from above are adapted to include the re-centering:
@inline function g_star(w,z,theta, g_center) 
    return (theta[1] * w[1]^(-theta[2]) * w[2] - 1) .* z[:] - g_center
end

function gn_star(w,z,theta, g_center)    
    1/size(w)[1]*(sum((theta[1] * w[:,1] .^(-theta[2]) .* w[:,2] - 1) .* z, 1))[1,:] - g_center
end

function gn_wrapper_star(w,z,g_center)
    return theta -> gn_star(w,z,theta,g_center)
end

function What_star(w,z,theta, g_center) 
    result = zeros(Float64, size(z)[2], size(z)[2])
    for i in 1:size(w)[1]
        result .+= g_star(w[i,:],z[i,:],theta,g_center)*g_star(w[i,:],z[i,:],theta,g_center)'
    end
    return inv(result/size(z)[1])
end

function Qn_wrapper_star(w, z, g_center)
    return function f(theta) 
        v = gn_star(w,z,theta,g_center)
        return 1/2*(v'*v)[1,1]
    end
end

function Qn_wrapper_star(w, z, What, g_center)
    return function f(theta) 
        v = gn_star(w,z,theta,g_center)
        return 1/2*(v'*What*v)[1,1]
    end
end



Qn_wrapper_star (generic function with 2 methods)

In [24]:
#draws from block bootstrap, calculates θ, aVar and returns t-statistics
function bootstrap_draw(w,z,θhat,length=5)
    n = size(w)[1] #number of rows
    no_blocks = ceil(Integer,n/length) #number of blocks in the original data
    sel_blocks = ceil(Integer,rand(Uniform(),50)*no_blocks) #randomly selected blocks
    
    #prepare new matrices:
    i = 1
    w_perm = w[length*(sel_blocks[i]-1)+1:min(n,length*sel_blocks[i]),:]
    z_perm = z[length*(sel_blocks[i]-1)+1:min(n,length*sel_blocks[i]),:]
    
    for i in 2:50
        w_perm = vcat(w_perm, w[length*(sel_blocks[i]-1)+1:min(n,length*sel_blocks[i]),:])
        z_perm = vcat(z_perm, z[length*(sel_blocks[i]-1)+1:min(n,length*sel_blocks[i]),:])
    end
    
    n = size(w_perm)[1]
    
    #GMM
    
        g_center = gn(w, z, θhat) #g_center is calculated from the original data set
        Qn = Qn_wrapper_star(w_perm, z_perm, g_center)
        θfirst = optimize(Qn, θhat, BFGS(), Optim.OptimizationOptions(g_tol=1e-15, autodiff=true)).minimum
        Whatfirst = What_star(w_perm, z_perm, θfirst, g_center)
        
        Qnsecond = Qn_wrapper_star(w_perm, z_perm, Whatfirst, g_center) 
        θsecond = optimize(Qnsecond, θfirst, BFGS(), Optim.OptimizationOptions(g_tol=1e-15, autodiff=true)).minimum
        
        #aVar
        dgn_star(theta) = ForwardDiff.jacobian(gn_wrapper_star(w_perm,z_perm, g_center),theta)
        G = convert(Array{Float64,2},dgn_star(θsecond))
        aVar = inv(G'*What_star(w_perm,z_perm,θsecond, g_center)*G)
    
        tδ = (θsecond[1]-θhat[1])/sqrt(aVar[1,1]/n)
        tγ = (θsecond[2]-θhat[2])/sqrt(aVar[2,2]/n)

    return [tδ tγ]
end

#iterates the above function and calculates z_star
function z_star(w, z, B=100, length=5)
    
    i = 1
    boot_dist = bootstrap_draw(w,z,θhat,length)
    
    while size(boot_dist)[1]<B 
        try
            boot_dist = vcat(boot_dist, bootstrap_draw(w,z,θhat,length))
            catch y
        end
    end
    
    return [percentile(abs(boot_dist[:,1]),100-5), percentile(abs(boot_dist[:,2]),100-5)]
end

z_star (generic function with 3 methods)

In [25]:
@time zstar_boot5 = z_star(w, z, 1000, 5)

417.871041 seconds (223.97 M allocations: 10.679 GB, 1.24% gc time)


2-element Array{Float64,1}:
 1.96021
 1.98819

In [26]:
θhat[1] + sqrt(aVar[1,1]/n) * [-zstar_boot5[1], zstar_boot5[1]]

2-element Array{Float64,1}:
 0.938112
 1.01895 

In [27]:
θhat[2] + sqrt(aVar[2,2]/n) * [-zstar_boot5[2], zstar_boot5[2]]

2-element Array{Float64,1}:
 1.12568
 5.64632

c
--
For a block length of 2:

In [28]:
zstar_boot2 = z_star(w, z, 1000, 2)

2-element Array{Float64,1}:
 1.75665
 1.67427

In [29]:
θhat[1] + sqrt(aVar[1,1]/n) * [-zstar_boot2[1], zstar_boot2[1]]

2-element Array{Float64,1}:
 0.942309
 1.01475 

In [30]:
θhat[2] + sqrt(aVar[2,2]/n) * [-zstar_boot2[2], zstar_boot5[2]]

2-element Array{Float64,1}:
 1.48257
 5.64632

For a block length of 10:

In [31]:
zstar_boot10 = z_star(w, z, 1000, 2)

2-element Array{Float64,1}:
 1.82353
 1.75656

In [32]:
θhat[1] + sqrt(aVar[1,1]/n) * [-zstar_boot10[1], zstar_boot10[1]]

2-element Array{Float64,1}:
 0.94093
 1.01613

In [33]:
θhat[2] + sqrt(aVar[2,2]/n) * [-zstar_boot10[2], zstar_boot10[2]]

2-element Array{Float64,1}:
 1.38902
 5.38298

d
--
I report $\delta$ and $\gamma$ in columns:

In [34]:
[zstar_boot2, zstar_boot5, zstar_boot10]

3-element Array{Array{Float64,1},1}:
 [1.75665,1.67427]
 [1.96021,1.98819]
 [1.82353,1.75656]